Maxime Marchand
# Astrophysics and Data Science : Project 5
## Neural network from scratch, part 2
We now train the neural network on custom made 2D data points.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from functions import *

We implement a function that returns a boolean array to store the points located in a given circle.

In [ ]:
def points_in_circle(points, circle):
    """
    Selects points in a circle
    
    PARAMETERS
        points : np.ndarray [[x1, y1], [x2, y2], ..., [xN, yN]]
        circle : np.array   [x, y, radius]
    
    RETURNS
        Boolean array with True where the points are in the circle, False otherwise.
    """
    return (points[:, 0] - circle[0])**2 + (points[:, 1] - circle[1])**2 <= circle[2]**2

### Creation of the dataset

In [ ]:
num_points = 10000
points     = np.random.uniform(low=-1.0, high=1.0, size=(num_points, 2)) # Array of point coordinates
circles    = np.array([[-0.7,  0.5,  0.5],                               # Array of circles [[x, y, radius], ...]
                       [ 0.7,  0.5,  0.5],
                       [ 0.0,  0.5,  0.6]])

# Boolean array to store the points belonging to a circle
sel = np.ndarray(shape=(num_points, len(circles)), dtype=bool)

# Fill the sel variable
for _ in range(circles.shape[0]):
    sel[:, _] = points_in_circle(points, circles[_])
    
classes = np.sum(sel, axis=1, dtype=bool)  # 0 : point not in circle, 1 : Point in circle

## Data visualisation

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
# Plot all the points
ax.scatter(points[:, 0], points[:, 1], s=3) 
# Plot the points located in circles
[ax.scatter(points[:, 0][sel[:, _]], points[:, 1][sel[:, _]], s=3, c='C1') for _ in range(sel.shape[1])];

### Preparing data for training and validation

In [ ]:
# Creating the targets (converting 0 -> [1, 0] and 1 -> [0, 1])
targets = np.ndarray(shape=(len(classes), 2))
targets[classes==False, :] = np.array([1, 0])
targets[classes==True , :] = np.array([0, 1])

# Splitting the data for training and validation
training_percent = int(num_points * 0.8) # 80% for training

training_set = {
    'vectors' :  points[:training_percent],
    'classes' : classes[:training_percent],
    'targets' : targets[:training_percent]
}

validation_set = {
    'vectors' :  points[training_percent:],
    'classes' : classes[training_percent:],
    'targets' : targets[training_percent:]
}

### Training the network

In [ ]:
# Dimention of the data
class_dim = len(np.unique(training_set['classes']))  # Number of possible outputs (2 in the case of points)
data_dim  = len(training_set['vectors'][0])          # Dimention of the data (2 in the case of points)

# Architecture of the network
num_neurons_input  = data_dim
num_hidden_layers  = 0
num_neurons_hidden = 2
num_neurons_output = class_dim

# Training parameters
num_epochs     = 3
learning_rate  = 0.05
transfert_mode = 'sig'

network = generate_neural_network(num_neurons_input, num_hidden_layers, num_neurons_hidden, num_neurons_output)
network, vec_cost = train_network(network, num_epochs, learning_rate, training_set, transfert_mode)

In [ ]:
plt.plot(vec_cost)

In [ ]:
def class_from_output(output):
    """
    Converts target vector to class
    
    PARAMETERS
        output : np.array
        
    RETURNS
        0 if output == [1, 0]
        1 if output == [0, 1]
    """
    return float(np.where(output == np.max(output))[0])

In [ ]:
predicted_values = np.ndarray(len(validation_set['vectors']))

for i in range(len(validation_set['vectors'])):
    entry      = validation_set['vectors'][i]                        # Data to put in network
    network    = forward_propagation(network, entry, transfert_mode) # Forward propagation 
    net_output = network[-1]['O']                                    # Predicted value from network
    predicted_values[i] = class_from_output(net_output)              # Storing the predicted class
    
    
# Counting good answers
numGoodAnswers = 0
for i in range(len(predicted_values)):
    if predicted_values[i] == validation_set['classes'][i]:
        numGoodAnswers += 1
        
goodAnswersPercent = 100*numGoodAnswers/len(predicted_values)
print(numGoodAnswers, "/", len(predicted_values), "good answers. ({:.2f}%)".format(goodAnswersPercent))

if goodAnswersPercent > 90:
    print("Yahoo !")